In [1]:
import pandas as pd
import re
from data_extraction import DataExtractor, DatabaseConnector
from database_utils import DatabaseConnector
from data_cleaning import DataCleaning


Cleaning and uploading user data

In [2]:
RDS_CONNECTOR = DatabaseConnector()
RDS_CONNECTOR.init_db_engine()
RDS_CONNECTOR.list_db_tables()

lamb = DataExtractor()
# ['legacy_store_details', 'legacy_users', 'orders_table']
# print(lamb.read_rds_table('legacy_users'))

## User Data
retrieved_user = lamb.read_rds_table('legacy_users')
test_clean = DataCleaning()
cleaned_user = test_clean.clean_user_data(retrieved_user)
RDS_CONNECTOR.upload_to_db(cleaned_user, 'dim_users')

['legacy_store_details', 'legacy_users', 'orders_table']


Cleaning and uploading card data

In [3]:
## Card Data
retrieved_data = lamb.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')
#print(retrieved_data)
cleaned_card_data = test_clean.clean_card_data(retrieved_data)
RDS_CONNECTOR.upload_to_db(cleaned_card_data, 'dim_card_details')

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Cleaning and uploading card data

In [4]:
## Card data
api_key = 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'
endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'

headers = {'x-api-key': api_key}

number_of_stores = lamb.list_number_of_stores(endpoint, headers)

other = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
stores_data = lamb.retrieve_stores_data(other, headers, number_of_stores)
cleaned_stores_data = test_clean.clean_store_data(stores_data)
RDS_CONNECTOR.upload_to_db(cleaned_stores_data, 'dim_store_details')

Status Code: 200
API Response: {'statusCode': 200, 'number_stores': 451}
     index                                            address  longitude  \
0        0                                                N/A        NaN   
1        1  Flat 72W, Sally isle, East Deantown, E7B 8EB, ...   51.62907   
2        2        Heckerstraße 4/5, 50491 Säckingen, Landshut   48.52961   
3        3  5 Harrison tunnel, South Lydia, WC9 2BE, Westbury   51.26000   
4        4  Studio 6, Stephen landing, South Simon, B77 2W...   53.02330   
..     ...                                                ...        ...   
445    445  Flat 7, Stephanie lake, Morrisside, HP8 8LH, C...   50.76306   
446    446    Täschestraße 25, 39039 Nördlingen, Kirchlengern   52.20000   
448    448  Studio 8, Moss mall, West Linda, M0E 6XR, High...   51.62907   
449    449               Baumplatz 6, 80114 Kötzting, Bretten   49.03685   
450    450  Gotthilf-Rose-Straße 7/3, 45457 Feuchtwangen, ...   50.64336   

         local

Cleaning and uploading product data

In [5]:
### Product Data
s3_address_products = 's3://data-handling-public/products.csv'
extracted_product_data = lamb.extract_from_s3(s3_address_products)
cleaned_product_data = test_clean.clean_products_data(extracted_product_data)
RDS_CONNECTOR.upload_to_db(cleaned_product_data, "dim_products")

data-handling-public products.csv
s3://data-handling-public/products.csv


Cleaning and uploading orders data

In [6]:
## Order Data
retrieved_orders = lamb.read_rds_table('orders_table')
cleaned_orders = test_clean.clean_orders_data(retrieved_orders)
RDS_CONNECTOR.upload_to_db(cleaned_orders, 'orders_table')

In [7]:
print(retrieved_orders.columns)


Index(['level_0', 'index', 'date_uuid', 'first_name', 'last_name', 'user_uuid',
       'card_number', 'store_code', 'product_code', '1', 'product_quantity'],
      dtype='object')


In [8]:
## Time
s3_address_date = 'https://data-handling-public/date_details.json'
extracted_product_data = lamb.extract_from_s3(s3_address_date)

data-handling-public.s3.eu-west-1.amazonaws.com date_details.json
https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json


NoSuchBucket: An error occurred (NoSuchBucket) when calling the GetObject operation: The specified bucket does not exist